In [1]:
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')
from Utils.utils import create_URM, create_ICM, combine_matrices
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
##NON-PERSONALIZED
from src.Recommenders.Base.NonPersonalizedRecommender import TopPop

#KNN
from src.Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from src.Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender

#MATRIX-FACTORIZATION
from src.Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from src.Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from src.Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython, MatrixFactorization_AsySVD_Cython
from src.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from src.Recommenders.MatrixFactorization.IALSRecommenderLinear import IALSRecommender
from src.Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#GRAPHIC-BASED
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

#HYBRID
from src.Hybrids.MergingModelsByScores import MergeThreeModelsByScores

In [2]:
##CREATION OF MATRICES AND SPLITTING IN TRAIN-TEST-VALIDATION
URM = create_URM()
ICM = create_ICM()
URM_stacked = combine_matrices(ICM=ICM, URM=URM)

In [3]:
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_stacked, train_percentage=0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage=0.85)

In [25]:
import numpy as np 
import scipy.sparse as sps

profile_length = np.ediff1d(sps.csr_matrix(URM).indptr)
profile_length, profile_length.shape

(array([168, 609, 140, ..., 260, 182, 329], dtype=int32), (13650,))

In [26]:
block_size = int(len(profile_length)*0.20)
block_size

2730

In [27]:
sorted_users = np.argsort(profile_length)
sorted_users

array([1876, 9021, 5660, ..., 1141, 8526,  317])

In [28]:
len(sorted_users)

13650

In [29]:
cold_start_pos = 0
cold_end_pos = block_size
cold_users_group = sorted_users[cold_start_pos:cold_end_pos]

In [30]:
len(cold_users_group)

2730

In [31]:
warm_users_group = sorted_users[cold_end_pos:]

In [32]:
len(warm_users_group)

10920

In [51]:
class TopPopRecommender(object):

    def fit(self, URM_train):
        
        self.URM_train = URM_train

        item_popularity = np.ediff1d(URM.tocsc().indptr)

        # We are not interested in sorting the popularity value,
        # but to order the items according to it
        self.popular_items = np.argsort(item_popularity)
        self.popular_items = np.flip(self.popular_items, axis = 0)
    
    
    def recommend(self, user_id, at=10, remove_seen=True):

        if remove_seen:
            seen_items = self.URM_train.indices[self.URM_train.indptr[user_id]:self.URM_train.indptr[user_id+1]]
            
            unseen_items_mask = np.in1d(self.popular_items, seen_items,
                                        assume_unique=True, invert = True)

            unseen_items = self.popular_items[unseen_items_mask]

            recommended_items = unseen_items[0:at]

        else:
            recommended_items = self.popular_items[0:at]
            

        return recommended_items

In [52]:
##TOPPOP
TopPop_recommender = TopPopRecommender()
TopPop_recommender.fit(URM)

In [18]:
### RP3Beta Recommender
RP3Beta_recommender = RP3betaRecommender(URM_train=URM_stacked)
RP3Beta_recommender.fit(topK=181, alpha=0.5709402717259106, beta=0.3566066452521787, implicit=True, normalize_similarity=True)

## IALS Recommender
IALS_recommender = IALSRecommender(URM_train=URM_stacked)
IALS_recommender.fit(epochs=50, num_factors=40, alpha=0.7917156202136513, reg=1.0)

##SLIMELASTICNET Recommender
SLIME_recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=URM_stacked)
SLIME_recommender.fit(topK=420, l1_ratio=0.06061216785568925, alpha=0.00554982938879437, workers = 10)

IALSRecommenderLinear: Epoch 1 of 50. Elapsed time 6.43 sec
IALSRecommenderLinear: Epoch 2 of 50. Elapsed time 12.78 sec
IALSRecommenderLinear: Epoch 3 of 50. Elapsed time 19.21 sec
IALSRecommenderLinear: Epoch 4 of 50. Elapsed time 25.52 sec
IALSRecommenderLinear: Epoch 5 of 50. Elapsed time 31.95 sec
IALSRecommenderLinear: Epoch 6 of 50. Elapsed time 38.21 sec
IALSRecommenderLinear: Epoch 7 of 50. Elapsed time 45.07 sec
IALSRecommenderLinear: Epoch 8 of 50. Elapsed time 51.36 sec
IALSRecommenderLinear: Epoch 9 of 50. Elapsed time 57.71 sec
IALSRecommenderLinear: Epoch 10 of 50. Elapsed time 1.07 min
IALSRecommenderLinear: Epoch 11 of 50. Elapsed time 1.17 min
IALSRecommenderLinear: Epoch 12 of 50. Elapsed time 1.28 min
IALSRecommenderLinear: Epoch 13 of 50. Elapsed time 1.39 min
IALSRecommenderLinear: Epoch 14 of 50. Elapsed time 1.55 min
IALSRecommenderLinear: Epoch 15 of 50. Elapsed time 1.66 min
IALSRecommenderLinear: Epoch 16 of 50. Elapsed time 1.76 min
IALSRecommenderLinear: Ep

100%|████████████████████████████████████▉| 18056/18059 [14:00<00:00, 22.37it/s]

In [20]:
hybrid_recommenders = [RP3Beta_recommender, IALS_recommender, SLIME_recommender]

##SLIME_RP3BETA_IALS
SLIM_RP3_IALS_recommender = MergeThreeModelsByScores(URM_stacked,hybrid_recommenders)
SLIM_RP3_IALS_recommender.fit(alpha=0.0, beta=0.3023745211799622, gamma=1.0)



In [53]:
submission_cold =[]
for user in cold_users_group:
    submission_cold.append((user,TopPop_recommender.recommend(user,10)))

In [54]:
submission_warm =[]
for user in warm_users_group:
    submission_warm.append((user,SLIM_RP3_IALS_recommender.recommend(user,10)))

In [55]:
len(submission_cold)

2730

In [56]:
len(submission_warm)

10920

In [57]:
submission = submission_cold + submission_warm

In [58]:
len(submission)

13650

In [64]:
print(submission[10])

(10, [3899, 14358, 6214, 6975, 14400, 16365, 13229, 4607, 10954, 13410])


In [60]:
submission.sort()

In [65]:
print(submission[0])

(0, array([ 3899,  8635, 16365,  2665, 13134, 15778, 13914, 13229,  3768,
        1551]))


In [66]:
for i in range(0,13650):
    print(submission[i])

(0, array([ 3899,  8635, 16365,  2665, 13134, 15778, 13914, 13229,  3768,
        1551]))
(1, [3473, 940, 14400, 2118, 16012, 9275, 10118, 640, 9373, 16365])
(2, array([ 6177, 17330, 16365, 15119,  5273, 14181, 16846, 10865,  2665,
       13134]))
(3, array([ 3899,  3473,  6214,  8635, 17330, 16365, 15119,  5273, 14181,
       14358]))
(4, [8194, 10586, 16494, 9871, 10954, 11836, 2152, 6177, 5299, 13691])
(5, array([ 3899,  3473,  6214,  8635, 17330, 16365, 15119,  5273, 14181,
       14358]))
(6, array([ 6214,  5273, 14181, 14358, 16846, 13134,  5925, 13229,  3768,
        1551]))
(7, [13963, 15119, 3433, 1885, 14031, 7751, 4292, 15392, 17364, 640])
(8, [8635, 5605, 3619, 2665, 1259, 10865, 5299, 3768, 4853, 12752])
(9, [5605, 8207, 1614, 13318, 14181, 14078, 15117, 3473, 1551, 3347])
(10, [3899, 14358, 6214, 6975, 14400, 16365, 13229, 4607, 10954, 13410])
(11, [14181, 3473, 11359, 10865, 5577, 6431, 5337, 12390, 11776, 10370])
(12, [3899, 6724, 3200, 1137, 1792, 13963, 15778, 17140, 

(2028, [7264, 8194, 447, 4582, 8911, 2301, 901, 16364, 3768, 14181])
(2029, [1846, 7744, 7867, 10243, 53, 6177, 14868, 3591, 17330, 1603])
(2030, array([ 3899,  5273, 14181, 16846, 10865, 13134, 15778,  5925, 13914,
       13229]))
(2031, [17788, 16846, 3768, 4607, 3473, 6589, 12752, 2152, 1782, 6035])
(2032, [17135, 4607, 1619, 6597, 1137, 640, 3889, 5433, 7591, 5273])
(2033, array([ 6177,  3899,  3473,  6214, 17330, 16365, 15119,  5273, 14181,
       14358]))
(2034, [12371, 2545, 2126, 16291, 13417, 7919, 15583, 4396, 838, 6354])
(2035, [16846, 5403, 6214, 5273, 16946, 15119, 6589, 10954, 17830, 14358])
(2036, [17520, 10370, 8635, 4089, 5403, 1005, 7295, 11379, 287, 15750])
(2037, array([ 3899,  3473,  6214,  8635, 17330, 16365,  5273, 14181, 14358,
        2665]))
(2038, array([ 6177,  3899,  6214, 14181, 14358, 16846,  5925, 13229, 14868,
        8207]))
(2039, [13229, 14095, 17356, 13134, 6670, 6724, 1259, 17936, 1792, 9871])
(2040, [2179, 14996, 12028, 17520, 13318, 3942, 17961, 

(4027, [8011, 6214, 3200, 15639, 17416, 16549, 1941, 17748, 1137, 1434])
(4028, [17584, 8651, 3200, 13563, 1259, 11776, 15683, 11481, 4088, 17788])
(4029, [14868, 17330, 249, 16598, 14181, 17022, 9610, 9441, 940, 16332])
(4030, array([ 6214, 17330, 16365,  5273, 14358, 16846, 10865, 13134, 15778,
       13914]))
(4031, [1551, 1846, 14181, 14396, 409, 11227, 6151, 5951, 388, 16486])
(4032, [3473, 7257, 8011, 15119, 16486, 17788, 2307, 16549, 12371, 8573])
(4033, [8194, 4292, 15082, 12186, 14358, 3899, 13963, 9482, 16846, 1137])
(4034, [17788, 3473, 6214, 12901, 2886, 6589, 9130, 6751, 15551, 16624])
(4035, [3473, 4607, 15119, 8194, 1941, 7870, 13229, 17294, 9131, 17584])
(4036, [5299, 6214, 14181, 14400, 16791, 1137, 14226, 4645, 2243, 2152])
(4037, [1232, 10562, 8651, 13914, 5925, 3942, 15119, 6214, 6905, 8326])
(4038, array([ 6177,  8635, 15119, 16846,  2665, 13229,  3768,  1551,  1259,
        1941]))
(4039, [13229, 17364, 12028, 12186, 14781, 17284, 1614, 5433, 16791, 5273])
(4040, 

        2665]))
(6194, [14358, 17364, 2152, 6011, 15119, 640, 17584, 8207, 5324, 8651])
(6195, array([ 6177,  3899, 16365, 15119,  5273, 14358, 16846, 10865,  2665,
        5925]))
(6196, array([ 3899,  8635, 17330, 16365,  5273, 14181, 14358, 13134, 15778,
       13914]))
(6197, array([16365, 10865,  2665, 13134,  5925, 13914, 13229,  6975,  3768,
        1551]))
(6198, [17416, 1792, 14781, 15119, 17666, 12360, 8635, 14037, 14833, 56])
(6199, [13229, 3200, 8207, 10562, 8371, 1259, 6873, 12186, 16365, 9131])
(6200, array([ 3899,  8635, 17330, 10865, 15778,  5925, 13229,  3768,  1259,
        1941]))
(6201, [99, 17623, 10865, 12764, 15119, 5307, 14449, 11504, 409, 1255])
(6202, array([ 3899,  3473,  6214,  8635, 17330, 15119,  5273, 14181, 14358,
       10865]))
(6203, [16808, 3200, 16365, 4292, 6340, 6652, 15181, 9131, 5742, 3899])
(6204, [249, 4607, 17294, 14917, 4292, 14400, 7870, 16170, 3433, 2552])
(6205, [10562, 12492, 17330, 16365, 5925, 8502, 12457, 2886, 7983, 11339])
(6206, [8

(8474, [6177, 7867, 5403, 17364, 15119, 13410, 1551, 4204, 12186, 16791])
(8475, array([ 3473,  6214, 16365, 15119,  5273, 14181, 14358, 16846, 10865,
       13134]))
(8476, [17936, 15778, 3473, 16070, 13178, 6782, 1137, 9322, 14358, 14400])
(8477, [2772, 6214, 16364, 11240, 7295, 512, 13554, 4088, 5679, 6652])
(8478, [17135, 2772, 7760, 13914, 1386, 8502, 16365, 14213, 14358, 16946])
(8479, [3899, 17135, 1044, 3829, 9871, 6306, 17936, 1137, 15969, 16012])
(8480, [15746, 8747, 13725, 8635, 10865, 16012, 9619, 12752, 10118, 11836])
(8481, array([ 3899,  3473,  6214,  8635, 17330, 15119,  5273, 14181, 14358,
       16846]))
(8482, [15119, 13237, 1941, 10865, 16846, 1259, 17330, 13410, 6306, 8326])
(8483, [3889, 15778, 241, 2055, 8747, 15996, 16624, 10865, 17330, 3899])
(8484, [17330, 3829, 14868, 1614, 6652, 14485, 5640, 10295, 17411, 4204])
(8485, [15119, 8207, 14358, 17520, 6214, 3899, 16791, 4853, 17294, 1619])
(8486, [13654, 13549, 11339, 1653, 4078, 11303, 5925, 14449, 11240, 6540])

(10526, [13134, 14571, 3768, 6214, 3668, 14358, 10865, 13410, 2486, 5299])
(10527, [552, 16640, 6392, 13585, 13229, 5312, 99, 10865, 12752, 4607])
(10528, [640, 2665, 10370, 6011, 8011, 1792, 12752, 1551, 9131, 2836])
(10529, [10370, 14996, 10562, 3942, 10865, 11339, 5585, 8247, 5742, 985])
(10530, array([ 6214, 17330, 16365, 15119,  5273, 14181, 14358, 16846,  2665,
       13229]))
(10531, [17330, 640, 2772, 16070, 5433, 8011, 2665, 17788, 8207, 5141])
(10532, [16364, 3200, 9322, 7295, 3768, 7486, 1619, 14358, 17294, 12356])
(10533, [17330, 8502, 16070, 3768, 6011, 16791, 5403, 5299, 17022, 9131])
(10534, [13914, 1933, 15119, 6975, 512, 15750, 640, 14396, 16021, 15181])
(10535, array([ 3899, 16365, 14358, 13134, 15778, 13914, 13229,  3768,  1551,
        1259]))
(10536, [5100, 6118, 16359, 4582, 9825, 8207, 12514, 6214, 13001, 12944])
(10537, array([ 3899,  3473, 17330, 16365, 15119, 14181, 14358, 16846, 10865,
        2665]))
(10538, array([ 3899,  6214, 14181, 10865,  2665, 13134, 1

       13134]))
(12026, [8580, 15473, 7744, 5951, 1114, 12752, 782, 11796, 11343, 15426])
(12027, [6589, 3768, 6011, 12186, 6177, 16846, 16365, 50, 8207, 3569])
(12028, [13950, 3815, 1551, 8502, 6730, 241, 7309, 5273, 17894, 13703])
(12029, [3473, 5847, 11491, 6724, 3200, 5640, 129, 955, 3636, 10370])
(12030, [13229, 2665, 4292, 2772, 5742, 17294, 12186, 4204, 17022, 5300])
(12031, [5273, 9871, 6214, 8502, 13229, 13725, 16070, 640, 4088, 15750])
(12032, [9131, 11973, 7974, 3768, 15778, 5403, 7975, 14229, 10865, 13549])
(12033, [6177, 2251, 6975, 8936, 13134, 11339, 3200, 17330, 14181, 6011])
(12034, [3200, 8567, 5273, 13229, 3473, 6214, 1259, 8635, 3619, 8651])
(12035, [7716, 9255, 1605, 17411, 8635, 6157, 14358, 17364, 8936, 9131])
(12036, array([15119, 10865, 15778, 13914, 13229,  6975,  1551,  1259, 14868,
        1137]))
(12037, [5273, 17022, 1259, 14868, 13638, 17526, 3473, 13691, 10865, 2456])
(12038, [16946, 14358, 17364, 13410, 12899, 4582, 6177, 5299, 3473, 1232])
(12039, arra

In [68]:
from Utils.utils import write_submission

write_submission(submission, name="coldTopPop_warmHybrid")